## Geospatial Data Scientist Census Activity
**Author:** Nicole Pepper

**Date:** 11/21/2025

**GitHub Repo:** https://github.com/nicolelpepper/af-census-activity

### Set Up Workspace

In [1]:
# ---- Load libraries ----
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
import re

### Load Project Data

In [2]:
# ---- Read in project data ----

# american community survey data
acs = pd.read_csv("data/raw/ACSDP5Y2023.DP05_2025-11-17T173744/ACSDP5Y2023.DP05-Data.csv")

# landcover data
landcover = rasterio.open("data/raw/landcover/landcover_conus.tif")

# schools data
private_schools = gpd.read_file("data/raw/schools/EDGE_GEOCODE_PRIVATESCH_2122/EDGE_GEOCODE_PRIVATESCH_2122.shp")
public_schools = gpd.read_file("data/raw/schools/EDGE_GEOCODE_PUBLICSCH_2324/EDGE_GEOCODE_PUBLICSCH_2324.shp")

# census tracts data for 2025 (choose a state)
florida_tracts = gpd.read_file("data/raw/tracts/12/tl_2025_12_tract.shp")
# oregon_tracts = gpd.read_file("data/raw/tracts/41/tl_2025_41_tract.shp")
# pennsylvania_tracts = gpd.read_file("data/raw/tracts/42/tl_2025_42_tract.shp")

/var/folders/f1/_qnbjphx7mvb9p2c19t_s1540000gn/T/ipykernel_53718/2079468737.py:4: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,238,246,254,324,338,372) have mixed types. Specify dtype option on import or set low_memory=False.
  acs = pd.read_csv("data/raw/ACSDP5Y2023.DP05_2025-11-17T173744/ACSDP5Y2023.DP05-Data.csv")


### Clean Data

In [ ]:
# --- Explore  & prep american community survey data ----

# Check column names
acs.columns

# Get snapshot of data
acs.head

# Save 1st row as metadata for columns (bc there are two header rows)
metadata_row = acs.iloc[0]
col_metadata = metadata_row.to_dict()
print(col_metadata)

# Drop extra header and reset index
acs = acs.drop(index=0).reset_index(drop=True)
acs.head

# ---- Select demographic data for Florida ----

# Select primary id fields and 2 demographic statistics of choice: 0037PE (percent white) and 0075PE (percent hispanic)
acs = acs[["GEO_ID", "NAME", "DP05_0001E","DP05_0037PE", "DP05_0075PE"]] 

# filter to Florida
fl_acs = acs[acs["GEO_ID"].str.contains("1400000US12")]

# ---- Clean errors in population statistics columns ----

# Select population statistics columns
pop_cols = ["DP05_0001E","DP05_0037PE","DP05_0075PE"]

# Force non-numeric values to NA
fl_acs[pop_cols] = fl_acs[pop_cols].apply(
    pd.to_numeric,
    errors="coerce")

fl_acs

{'geo_id': '1400000US12001000201', 'name': 'Census Tract 2.01; Alachua County; Florida', 'dp05_0001e': '5187', 'dp05_0037pe': '72.5', 'dp05_0075pe': '5187'}


KeyError: "None of [Index(['GEO_ID', 'NAME', 'DP05_0001E', 'DP05_0037PE', 'DP05_0075PE'], dtype='object')] are in the [columns]"